##**Deep Neural Networks for MNIST Classification**

Pipeline ▶
  - Prepare/Preprocess Data ▶ Split Data ▶ Train, Validate, Test.
  - Outline model, choose activation functions.
  - Set the appropriate advanced optimizers and loss function.
  - Make it learn/Train.
  - Test accuracy of the model.

In [1]:
# import relevant packages
import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds

### Acquire Data

In [2]:
# load dataset 
# as_supervised, loads into two tuple structure [input, target]
# with_info, provides tuple with info about version, features, number of samples
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
# look at dataset
mnist_dataset

{'test': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [4]:
# get info
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

### Prepare Data

In [5]:
mnist_train = mnist_dataset['train']
mnist_test = mnist_dataset['test']

In [6]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# convert num_validation_samples into an integer
num_validation_samples = tf.cast(num_validation_samples, tf.int64) 

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64) 

In [7]:
# create a function to scale our data

# the function below replicates a tensorflow function dataset.map(*function*)

def scale(image, label):
  '''
  This function takes in an image and label, converts them to floats, scales them on a scale from 0 to 1 
  by dividing by 255 (0 to 256, the # of shades of gray)
  '''
  image = tf.cast(image, tf.float32)
  image /= 255. # signifies we want image to be float

  return image, label


In [8]:
scaled_train_and_validation_data = mnist_train.map(scale)

scaled_test_data = mnist_test.map(scale)

**Shuffle Data**

In [9]:
# shuffle data for algorithm
# shuffle 10_000 values at once
BUFFER_SIZE = 10_000


shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

**Batching**
- Why? To increment performance and decrease memory usage

In [10]:
BATCH_SIZE = 100

# assign batch size for backpropagation, also creates a column in the tensor 
train_data = train_data.batch(BATCH_SIZE)

# assign validate and test data as one whole batch
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

### Model

Outline Model

In [12]:
# dimension of data
input_size = 28*28*1
# number of targets (0 to 9)
output_size = 10
# width of nn, number of neurons
hidden_layer_size = 50

model_1 = tf.keras.Sequential([
                               # flatten into one dimensional layer
                               tf.keras.layers.Flatten(input_shape=(28,28,1)),

                               # assess first hidden layer
                               tf.keras.layers.Dense(hidden_layer_size, activation='relu'),

                               # assess second hidden layer
                               tf.keras.layers.Dense(hidden_layer_size, activation='relu'),

                               # address final output layer, use softmax since later must transform data to probability
                               tf.keras.layers.Dense(output_size, activation='softmax')
                               ])

Choose Optimizer and Loss Function

In [13]:
# choose optimizer and loss
# sparse categorical crossentropy for data not yet one hot encoded (will apply one hot encoding)
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

Training

In [15]:
# assign number of epochs
num_epochs = 5

# fit model
model_1.fit(train_data, epochs=num_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.4122 - accuracy: 0.8811 - val_loss: 0.2048 - val_accuracy: 0.9398 - 8s/epoch - 15ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1808 - accuracy: 0.9469 - val_loss: 0.1547 - val_accuracy: 0.9513 - 4s/epoch - 7ms/step
Epoch 3/5
540/540 - 4s - loss: 0.1396 - accuracy: 0.9584 - val_loss: 0.1259 - val_accuracy: 0.9593 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 4s - loss: 0.1149 - accuracy: 0.9664 - val_loss: 0.1107 - val_accuracy: 0.9642 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0989 - accuracy: 0.9701 - val_loss: 0.0898 - val_accuracy: 0.9713 - 4s/epoch - 7ms/step


In [ ]:
# try with hidden layer size 100